# 自製智能中文選字系統  (1)

In [1]:
import sys
sys.version

'3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]'

## 資料前處理

確認版本為 python3

In [2]:
import re

In [3]:
def prepocess_line(line):
    # 僅僅挑出中文字元，並且斷開不連續的中文字
    # YOUR CODE HERE
    line_clean= re.sub(r'[^\u4E00-\u9FCC]', ' ', line)
    segments= line_clean.strip().split(' ')
    segments = [seg for seg in segments if seg]
    # END YOUR CODE
    return segments

In [4]:
prepocess_line('“英語”一詞源於遷居英格蘭的日耳曼部落盎格魯（），而“盎格魯”得名於')  
# 應該為：['英語', '一詞源於遷居英格蘭的日耳曼部落盎格魯', '而', '盎格魯', '得名於']

['英語', '一詞源於遷居英格蘭的日耳曼部落盎格魯', '而', '盎格魯', '得名於']

In [5]:
segments = []
with open('./wiki_zh_small.txt', encoding='UTF-8') as fr:
    for line in fr.readlines():
        segments += prepocess_line(line)

## Ngram

一開始要先計算字詞出現的次數

In [18]:
from collections import Counter

class Counters:
    def __init__(self, n):
        self.n = n
        self.counters = [Counter() for _ in range(n + 1)]  # 分別代表計算0、1、...個字的出現次數

    def fit(self, segments):
        # 因為 self.counters 分別代表計算0、1、...個字的出現次數
        # 請在此實作利用 segments 以及函式 _skip 來統計次數
        # YOUR CODE HERE
        for segment in segments:
            self.counters[0].update(['']*len(segment))
            for i in range(1,self.n+1):
                self.counters[i].update(self._skip(segment, i))
        # END YOUR CODE

    def __getitem__(self, k):
        return self.counters[k]

    def _skip(self, segment, n):
        assert n > 0
        if len(segment) < n:
            return []
        shift = n - 1
        for i in range(len(segment) - shift):
            yield segment[i:i+shift+1]

In [19]:
counters = Counters(n=3)
counters.fit(segments)

In [20]:
counters[0]
# 應該為： Counter({'': 371373})

Counter({'': 371370})

In [21]:
class Ngram:
    def __init__(self, n, counters):
        assert n <= counters.n
        self.n = n
        self.major_counter = counters[n]
        self.minor_counter = counters[n-1]

    def predict_proba(self, prefix='', top_k=5):
        assert len(prefix) >= self.n - 1
        # 使用 Ngram 的公式計算出下一個字出現的機率
        # 輸出為機率與字的tuple列表，詳見下方輸出範例
        # YOUR CODE HERE
        prefix = prefix[-(self.n-1):] if self.n > 1 else ''
        prefix_counts = self.minor_counter.get(prefix,0)
        candidates = []
        for k, v in self.major_counter.items():
            if k.startswith(prefix):
                candidates.append((v/prefix_counts, k[-1]))
                
        sorted_probs = sorted(candidates, key=lambda x: x[0], reverse=True)
        # END YOUR CODE
        return sorted_probs[:top_k] if top_k > 0 else sorted_probs

    def get_proba_dict(self, prefix=''):
        return {word: prob for prob, word in self.predict_proba(prefix, top_k=-1)}


In [22]:
unigram = Ngram(1, counters)

In [23]:
unigram.predict_proba('我思')
# 應該為：[(0.035732269174118744, '的'),
#         (0.012927703414087723, '國'),
#         (0.010620050461395955, '中'),
#         (0.009984570768472667, '在'),
#         (0.009852627950874188, '一')]

[(0.035732557826426474, '的'),
 (0.012927807846621966, '國'),
 (0.010620136252255163, '中'),
 (0.00998465142580176, '在'),
 (0.009852707542343216, '一')]

In [24]:
bigram = Ngram(2, counters)
trigram = Ngram(3, counters)

## 使用Ngram來建立第一版選字系統

In [25]:
class ChineseWordRecommenderV1:
    def __init__(self, unigram, bigram, trigram):
        self.unigram = unigram
        self.bigram = bigram
        self.trigram = trigram
    
    def predict_proba(self, prefix='', top_k=5):
        # 使用Ngram來建立選字系統
        # YOUR CODE HERE
        if not prefix:
            return self.unigram.predict_proba(prefix, top_k)
        
        elif len(prefix) == 1:
            return self.bigram.predict_proba(prefix, top_k)
        
        else:
            return self.trigram.predict_proba(prefix, top_k)
        # END YOUR CODE

In [26]:
model = ChineseWordRecommenderV1(unigram, bigram, trigram)

In [27]:
probs = model.predict_proba('我思', top_k=10)
probs

[(0.75, '故'), (0.25, '維')]

## Demo

In [28]:
!pip install -U pip
!pip install -q ipywidgets

Script file 'D:\Anaconda3\Scripts\pip-script.py' is not present.
Script file 'D:\Anaconda3\Scripts\pip-script.py' is not present.


In [29]:
import ipywidgets as widgets

text = widgets.Textarea()
label = widgets.Label()
display(label, text)

def func(change):
    probs = model.predict_proba(change.new, top_k=10)
    label.value = ' ' + '\t'.join([word for prob, word in probs])

text.observe(func, names='value')

Label(value='')

Textarea(value='')